In [1]:
!pip install openeo
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.8/206.8 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 62.3 MB/s eta 0:00:00
  Attempting uninstall: xarray
    Found existing installation: xarray 2025.7.1
    Uninstalling xarray-2025.7.1:
      Successfully uninstalled xarray-2025.7.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 71.1 MB/s eta 0:00:00


In [2]:

import rasterio
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
import openeo

In [3]:
connection = openeo.connect("https://openeo.dataspace.copernicus.eu").authenticate_oidc()

Visit https://identity.dataspace.copernicus.eu/auth/realms/CDSE/device?user_code=TGYU-FZGI 📋 to authenticate.

✅ Authorized successfully

Authenticated using device code flow.


In [4]:
#Areas de interes
lago_atitlan = {
    "west": -91.326256,
    "east": -91.07151,
    "south": 14.5948,
    "north": 14.750979
}
lago_amatitlan = {
    "west": -90.638065,
    "east": -90.512924,
    "south": 14.412347,
    "north": 14.493799
}

In [8]:
from google.colab import drive
import os

# Montar Google Drive
drive.mount('/content/drive')

# Ruta dentro de Google Drive donde deseas guardar los archivos
output_dir = "/content/drive/MyDrive/Colab_Notebooks/images/Atitlan/"

# Crear la carpeta si no existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)  # Crear la carpeta si no existe

# Fechas con nubosidad menor a 20
fechas = [
    "2025-02-07", "2025-02-10", "2025-02-25", "2025-02-27", "2025-03-02", "2025-03-04",
    "2025-03-07", "2025-03-09", "2025-03-12", "2025-03-14", "2025-03-19", "2025-03-22",
    "2025-03-24", "2025-03-26", "2025-04-03", "2025-04-11", "2025-04-13", "2025-04-15",
    "2025-04-16", "2025-04-18", "2025-04-28", "2025-05-03", "2025-05-13", "2025-05-28",
    "2025-07-10", "2025-07-17", "2025-07-20", "2025-07-24", "2025-08-01"
]

# Iterar sobre cada fecha para cargar y descargar los datos
for fecha in fechas:
    # Cargar la colección para cada fecha específica
    atitlan_cube = connection.load_collection(
        "SENTINEL2_L2A",
        spatial_extent=lago_atitlan,
        temporal_extent=[fecha, fecha],  # Usar la fecha actual en el bucle
        bands=["B02","B03","B04", "B08"]
    )

    # Guardar y descargar los resultados para la fecha actual
    result_graph = atitlan_cube.save_result(format="GTIFF")
    job = connection.create_job(result_graph)
    job.start_and_wait()

    # Usar un nombre único para cada archivo descargado basado en la fecha
    output_file = f"{output_dir}Bandas_Atitlan_{fecha}.tif"
    job.download_results(output_file)


Mounted at /content/drive
0:00:00 Job 'j-250810033529446085233a250a4d7876': send 'start'
0:00:12 Job 'j-250810033529446085233a250a4d7876': created (progress 0%)
0:00:17 Job 'j-250810033529446085233a250a4d7876': created (progress 0%)
0:00:24 Job 'j-250810033529446085233a250a4d7876': created (progress 0%)
0:00:32 Job 'j-250810033529446085233a250a4d7876': created (progress 0%)
0:00:42 Job 'j-250810033529446085233a250a4d7876': running (progress N/A)
0:00:54 Job 'j-250810033529446085233a250a4d7876': running (progress N/A)
0:01:10 Job 'j-250810033529446085233a250a4d7876': running (progress N/A)
0:01:29 Job 'j-250810033529446085233a250a4d7876': running (progress N/A)
0:01:53 Job 'j-250810033529446085233a250a4d7876': finished (progress 100%)


/tmp/ipython-input-1123492004.py:40: UserDeprecationWarning: Call to deprecated method download_results. (Instead use `BatchJob.get_results` and the more flexible download functionality of `JobResults`) -- Deprecated since version 0.4.10.
  job.download_results(output_file)
/usr/local/lib/python3.11/dist-packages/openeo/rest/job.py:199: UserDeprecationWarning: Call to deprecated method get_result. (Use `BatchJob.get_results` instead.) -- Deprecated since version 0.4.10.
  return self.get_result().download_files(target)
/usr/local/lib/python3.11/dist-packages/openeo/rest/job.py:203: UserDeprecationWarning: Call to deprecated class _Result. (Use `JobResults` instead) -- Deprecated since version 0.4.10.
  return _Result(self)


0:00:00 Job 'j-2508100338144eeb82e656fd17dad1a9': send 'start'
0:00:12 Job 'j-2508100338144eeb82e656fd17dad1a9': created (progress 0%)
0:00:18 Job 'j-2508100338144eeb82e656fd17dad1a9': created (progress 0%)
0:00:24 Job 'j-2508100338144eeb82e656fd17dad1a9': created (progress 0%)
0:00:32 Job 'j-2508100338144eeb82e656fd17dad1a9': created (progress 0%)
0:00:42 Job 'j-2508100338144eeb82e656fd17dad1a9': created (progress 0%)
0:00:55 Job 'j-2508100338144eeb82e656fd17dad1a9': running (progress N/A)
0:01:10 Job 'j-2508100338144eeb82e656fd17dad1a9': running (progress N/A)
0:01:29 Job 'j-2508100338144eeb82e656fd17dad1a9': running (progress N/A)
0:01:54 Job 'j-2508100338144eeb82e656fd17dad1a9': running (progress N/A)
0:02:24 Job 'j-2508100338144eeb82e656fd17dad1a9': finished (progress 100%)
0:00:00 Job 'j-25081003405647b7b9ed228f7f3c86c6': send 'start'
0:00:12 Job 'j-25081003405647b7b9ed228f7f3c86c6': queued (progress 0%)
0:00:17 Job 'j-25081003405647b7b9ed228f7f3c86c6': queued (progress 0%)
0:00:

Descarga Imagenes - Amatitlan

In [11]:
# --- Montar Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

import os, glob, shutil
from datetime import datetime, timedelta

# Carpeta destino en tu Drive
output_dir = "/content/drive/MyDrive/Colab_Notebooks/images/Amatitlan/"
os.makedirs(output_dir, exist_ok=True)

# Fechas con nubosidad < 20 (puedes reusar tu misma lista o acortarla)
fechas = [
    "2025-02-07", "2025-02-10", "2025-02-25", "2025-02-27", "2025-03-02", "2025-03-04",
    "2025-03-07", "2025-03-09", "2025-03-12", "2025-03-14", "2025-03-19", "2025-03-22",
    "2025-03-24", "2025-03-26", "2025-04-03", "2025-04-11", "2025-04-13", "2025-04-15",
    "2025-04-16", "2025-04-18", "2025-04-28", "2025-05-03", "2025-05-13", "2025-05-28",
    "2025-07-10", "2025-07-17", "2025-07-20", "2025-07-24", "2025-08-01"
]

# BBOX de Amatitlán (ya lo definiste arriba)
# lago_amatitlan = {
#     "west":  -90.638065,
#     "east":  -90.512924,
#     "south": 14.412437,
#     "north": 14.493799
# }

COL = "SENTINEL2_L2A"  # importante: NO "Sentinel2"
BANDS = ["B02","B03","B04","B08"]

def fecha_mas_un_dia(s):
    return (datetime.strptime(s, "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")

for fecha in fechas:
    print(f"\n=== Procesando {fecha} (ventana {fecha} a {fecha_mas_un_dia(fecha)}) ===")
    try:
        # 1) Cargar colección (ventana de 1 día para evitar 0 resultados)
        amatitlan_cube = connection.load_collection(
            COL,
            spatial_extent=lago_amatitlan,
            temporal_extent=[fecha, fecha_mas_un_dia(fecha)],
            bands=BANDS
        )

        # 2) Exportar a GTIFF
        result_graph = amatitlan_cube.save_result(format="GTIFF")
        job = connection.create_job(result_graph)
        job.start_and_wait(show_error_logs=True)

        # 3) Descargar a carpeta temporal por fecha
        tmp_dir = f"/content/Amatitlan_tmp_{fecha}"
        os.makedirs(tmp_dir, exist_ok=True)
        job.download_results(target=tmp_dir)

        # 4) Tomar el .tif descargado y renombrar al nombre final
        geotiffs = glob.glob(os.path.join(tmp_dir, "*.tif"))
        if len(geotiffs) == 0:
            print(f"No se descargó ningún .tif para {fecha}. (¿sin datos?)")
        else:
            src = geotiffs[0]
            dst = os.path.join(output_dir, f"Bandas_Amatitlan_{fecha}.tif")
            shutil.move(src, dst)
            print(f"Guardado: {dst}")

        # 5) Limpieza de la carpeta temporal
        shutil.rmtree(tmp_dir, ignore_errors=True)

    except Exception as e:
        print(f"✗ Error en {fecha}: {e}")
        # Continúa con la siguiente fecha
        continue

print("\nListo. Revisa tu carpeta en Drive:", output_dir)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

=== Procesando 2025-02-07 (ventana 2025-02-07 a 2025-02-08) ===
0:00:00 Job 'j-2508100531094625ae1d1d9fc09d29d5': send 'start'
0:00:13 Job 'j-2508100531094625ae1d1d9fc09d29d5': created (progress 0%)
0:00:18 Job 'j-2508100531094625ae1d1d9fc09d29d5': created (progress 0%)
0:00:24 Job 'j-2508100531094625ae1d1d9fc09d29d5': created (progress 0%)
0:00:32 Job 'j-2508100531094625ae1d1d9fc09d29d5': created (progress 0%)
0:00:42 Job 'j-2508100531094625ae1d1d9fc09d29d5': created (progress 0%)
0:00:55 Job 'j-2508100531094625ae1d1d9fc09d29d5': created (progress 0%)
0:01:10 Job 'j-2508100531094625ae1d1d9fc09d29d5': running (progress N/A)
0:01:29 Job 'j-2508100531094625ae1d1d9fc09d29d5': running (progress N/A)
0:01:53 Job 'j-2508100531094625ae1d1d9fc09d29d5': running (progress N/A)
0:02:23 Job 'j-2508100531094625ae1d1d9fc09d29d5': finished (progress 100%)


/tmp/ipython-input-855666324.py:54: UserDeprecationWarning: Call to deprecated method download_results. (Instead use `BatchJob.get_results` and the more flexible download functionality of `JobResults`) -- Deprecated since version 0.4.10.
  job.download_results(target=tmp_dir)


Guardado: /content/drive/MyDrive/Colab_Notebooks/images/Amatitlan/Bandas_Amatitlan_2025-02-07.tif

=== Procesando 2025-02-10 (ventana 2025-02-10 a 2025-02-11) ===
0:00:00 Job 'j-250810053349403999f7d956ec03402f': send 'start'
0:00:12 Job 'j-250810053349403999f7d956ec03402f': created (progress 0%)
0:00:17 Job 'j-250810053349403999f7d956ec03402f': running (progress N/A)
0:00:24 Job 'j-250810053349403999f7d956ec03402f': running (progress N/A)
0:00:32 Job 'j-250810053349403999f7d956ec03402f': running (progress N/A)
0:00:42 Job 'j-250810053349403999f7d956ec03402f': running (progress N/A)
0:00:54 Job 'j-250810053349403999f7d956ec03402f': running (progress N/A)
0:01:10 Job 'j-250810053349403999f7d956ec03402f': running (progress N/A)
0:01:29 Job 'j-250810053349403999f7d956ec03402f': error (progress N/A)
Your batch job 'j-250810053349403999f7d956ec03402f' failed. Error logs:
[{'id': '[1754804065805, 382451]', 'time': '2025-08-10T05:34:25.805Z', 'level': 'error', 'message': 'create_pyramid faile

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

# Abrir el archivo multibanda
tif_path = "../data/Bandas_Atitlan_Completas.tif"

with rasterio.open(tif_path) as src:
    bandas = src.read()
    profile = src.profile  # Guardar metadata


In [ ]:
# Asignar bandas por índice (rasterio usa base-1)
B02 = bandas[0].astype(np.float32)  # Azul
B03 = bandas[1].astype(np.float32)  # Verde
B04 = bandas[2].astype(np.float32)  # Rojo
B05 = bandas[3].astype(np.float32)  # RedEdge 1
B07 = bandas[4].astype(np.float32)  # RedEdge 3
B08 = bandas[5].astype(np.float32)  # NIR
B8A = bandas[6].astype(np.float32)  # RedEdge 4
B11 = bandas[7].astype(np.float32)  # SWIR 1
B12 = bandas[8].astype(np.float32)  # SWIR 2


In [ ]:
# NDCI: (B05 - B04) / (B05 + B04)
ndci = np.where((B05 + B04) == 0, 0, (B05 - B04) / (B05 + B04))

# Modelo cúbico de clorofila-a
chl = 826.57 * ndci**3 - 176.43 * ndci**2 + 19 * ndci + 4.071


In [ ]:
plt.figure(figsize=(10, 6))
plt.imshow(chl, cmap='viridis')
plt.colorbar(label='Clorofila-a (estimada)')
plt.title('Cianobacteria - Lago Atitlán')
plt.axis('off')
plt.show()


In [ ]:
output_path = "../data/Atitlan_Chlorophyll_Map.tif"

with rasterio.open(
    output_path,
    'w',
    driver='GTiff',
    height=chl.shape[0],
    width=chl.shape[1],
    count=1,
    dtype=chl.dtype,
    crs=profile["crs"],
    transform=profile["transform"]
) as dst:
    dst.write(chl, 1)


Amatitlan


In [ ]:


amatitlan_cube = connection.load_collection(
    "SENTINEL2_L2A",
    spatial_extent=lago_amatitlan,
    temporal_extent=["2025-04-01", "2025-08-06"],  # ajusta fechas si quieres
    bands=["B02", "B03", "B04", "B05", "B07", "B08", "B8A", "B11", "B12"]
)

result_graph = amatitlan_cube.save_result(format="GTIFF")
job = connection.create_job(result_graph)
job.start_and_wait()
job.download_results("../data/Bandas_Amatitlan_Completas.tif")

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

tif_path = "../data/Bandas_Amatitlan_Completas.tif"  # <-- usa tu ruta real

with rasterio.open(tif_path) as src:
    arr = src.read()          # (bands, rows, cols)
    profile = src.profile     # para guardar luego con mismo CRS/transform

# Mapeo de bandas (mismo orden en que pedimos en la descarga):
B02 = arr[0].astype(np.float32)  # Azul
B03 = arr[1].astype(np.float32)  # Verde
B04 = arr[2].astype(np.float32)  # Rojo
B05 = arr[3].astype(np.float32)  # RedEdge 1
B07 = arr[4].astype(np.float32)  # RedEdge 3
B08 = arr[5].astype(np.float32)  # NIR
B8A = arr[6].astype(np.float32)  # RedEdge 4
B11 = arr[7].astype(np.float32)  # SWIR 1
B12 = arr[8].astype(np.float32)  # SWIR 2

# --- Cyano/Chl-a (como en evalscript.js) ---
# NDCI = (B05 - B04) / (B05 + B04)
ndci = np.where((B05 + B04) == 0, 0, (B05 - B04) / (B05 + B04))

# Clorofila-a (modelo cúbico del script)
chl = 826.57 * ndci**3 - 176.43 * ndci**2 + 19 * ndci + 4.071

# Visualizar
plt.figure(figsize=(10, 6))
plt.imshow(chl, cmap='viridis')
plt.colorbar(label='Clorofila-a (estimada)')
plt.title('Cianobacteria / Clorofila-a — Lago Amatitlán')
plt.axis('off')
plt.show()


In [ ]:
out_path = "../data/Amatitlan_Chlorophyll_Map.tif"

with rasterio.open(
    out_path,
    "w",
    driver="GTiff",
    height=chl.shape[0],
    width=chl.shape[1],
    count=1,
    dtype=chl.dtype,
    crs=profile["crs"],
    transform=profile["transform"]
) as dst:
    dst.write(chl, 1)

print("Guardado:", out_path)
